In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

DB_URL = os.getenv("DATABASE_URL")

engine = create_engine(DB_URL)

df = pd.read_sql("SELECT * FROM weather_data", engine)
df.head()


,id,city,timestamp,temperature,humidity,wind_speed,pm10,pm2_5,nitrogen_dioxide,ozone
0,1,Chennai,2025-11-12 18:47:43.629680+00:00,25.4,92.0,3.2,49.1,43.9,2.6,124.0
1,2,Bengaluru,2025-11-12 18:47:43.641602+00:00,19.3,93.0,4.2,45.9,44.7,12.8,75.0
2,3,Kolkata,2025-11-12 18:47:43.645636+00:00,17.4,95.0,3.4,135.5,125.3,21.0,54.0
3,4,Hyderabad,2025-11-12 18:47:43.647382+00:00,18.4,79.0,2.3,49.0,46.8,10.6,96.0
4,5,Mumbai,2025-11-12 18:47:43.654991+00:00,25.4,67.0,9.7,44.4,42.1,18.5,78.0


In [2]:
df.describe()

,id,temperature,humidity,wind_speed,pm10,pm2_5,nitrogen_dioxide,ozone
count,31.000000,31.000000,31.000000,31.000000,31.00000,31.000000,31.000000,31.000000
mean,16.000000,15.635484,80.774194,4.925806,50.63871,45.551613,14.590323,58.096774
std,9.092121,7.759920,14.983791,2.706285,36.02336,30.963569,9.808512,31.298940
min,1.000000,-6.700000,28.000000,1.800000,7.30000,4.800000,0.200000,6.000000
25%,8.500000,11.400000,75.000000,3.450000,22.45000,20.950000,8.550000,35.000000
50%,16.000000,15.900000,84.000000,4.000000,42.00000,40.500000,12.800000,58.000000
75%,23.500000,19.900000,90.500000,5.300000,65.55000,59.500000,19.750000,78.000000
max,31.000000,27.400000,98.000000,14.300000,135.50000,125.300000,42.400000,124.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                31 non-null     int64              
 1   city              31 non-null     object             
 2   timestamp         31 non-null     datetime64[ns, UTC]
 3   temperature       31 non-null     float64            
 4   humidity          31 non-null     float64            
 5   wind_speed        31 non-null     float64            
 6   pm10              31 non-null     float64            
 7   pm2_5             31 non-null     float64            
 8   nitrogen_dioxide  31 non-null     float64            
 9   ozone             31 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(7), int64(1), object(1)
memory usage: 2.6+ KB


In [4]:
df.isnull().sum()

id                  0
city                0
timestamp           0
temperature         0
humidity            0
wind_speed          0
pm10                0
pm2_5               0
nitrogen_dioxide    0
ozone               0
dtype: int64

In [5]:
df.nunique()

id                  31
city                31
timestamp           31
temperature         28
humidity            25
wind_speed          27
pm10                31
pm2_5               31
nitrogen_dioxide    28
ozone               27
dtype: int64

In [6]:
pd.read_sql("""
SELECT city , MIN(pm10) AS min_pm10
FROM weather_data
GROUP BY city
ORDER BY min_pm10
LIMIT 10
""", engine)


,city,min_pm10
0,Leh,7.3
1,Dehradun,12.3
2,Kavaratti,13.4
3,Shillong,15.4
4,Aizawl,15.8
5,Port Blair,17.2
6,Chandigarh,18.5
7,Itanagar,21.4
8,Kohima,23.5
9,Panaji,24.3


In [7]:
pd.read_sql("""
SELECT
    city,
    ROUND(AVG(temperature)::numeric) AS avg_temp
FROM weather_data
GROUP BY city
ORDER BY avg_temp DESC
LIMIT 10""", engine)


,city,avg_temp
0,Thiruvananthapuram,27.0
1,Port Blair,27.0
2,Kavaratti,27.0
3,Puducherry,25.0
4,Mumbai,25.0
5,Chennai,25.0
6,Panaji,23.0
7,Dispur,21.0
8,Bengaluru,19.0
9,Agartala,19.0


In [8]:
df_numeric = df.select_dtypes(include=['number'])
df_numeric.corr()

,id,temperature,humidity,wind_speed,pm10,pm2_5,nitrogen_dioxide,ozone
id,1.000000,-0.252572,-0.260091,0.213499,-0.390826,-0.422852,-0.146856,-0.142904
temperature,-0.252572,1.000000,0.360086,0.369231,0.081682,0.118671,-0.247473,0.278988
humidity,-0.260091,0.360086,1.000000,-0.175764,0.310342,0.331318,-0.199015,0.007653
wind_speed,0.213499,0.369231,-0.175764,1.000000,-0.318626,-0.352065,-0.381398,0.246318
pm10,-0.390826,0.081682,0.310342,-0.318626,1.000000,0.987849,0.221447,0.187486
pm2_5,-0.422852,0.118671,0.331318,-0.352065,0.987849,1.000000,0.275352,0.130686
nitrogen_dioxide,-0.146856,-0.247473,-0.199015,-0.381398,0.221447,0.275352,1.000000,-0.644055
ozone,-0.142904,0.278988,0.007653,0.246318,0.187486,0.130686,-0.644055,1.000000


In [9]:
city_avg = df.groupby('city')[['temperature', 'humidity', 'pm10', 'pm2_5', 'nitrogen_dioxide', 'ozone']].mean().reset_index()
city_avg

,city,temperature,humidity,pm10,pm2_5,nitrogen_dioxide,ozone
0,Agartala,19.1,91.0,78.5,73.8,18.1,47.0
1,Aizawl,11.1,96.0,15.8,15.1,13.4,6.0
2,Bengaluru,19.3,93.0,45.9,44.7,12.8,75.0
3,Bhopal,12.4,83.0,40.6,39.5,14.1,56.0
4,Bhubaneswar,15.6,90.0,90.3,84.5,22.4,33.0
5,Chandigarh,11.0,76.0,18.5,18.1,22.2,47.0
6,Chennai,25.4,92.0,49.1,43.9,2.6,124.0
7,Dehradun,7.8,74.0,12.3,12.2,26.9,6.0
8,Delhi,11.7,90.0,131.3,100.8,14.1,85.0
9,Dispur,20.5,94.0,42.0,40.5,9.8,42.0
